In [61]:
!conda create -y -n examenbd python=3.8

^C


In [1]:
!conda install -y pandas
!conda install -y requests
!conda install -y pathlib
!conda install -y -c conda-forge pyarrow,pyarrow

^C


# EJERCICIO 2

## Definición de métodos a utilizar

    - recorrer: recorre el json con los valores seleccionados adaptados en este caso

    - get_json: devuelve el json de una URL

    - get_apis_untilnext: recorre la api hasta que no haya más páginas

In [17]:
import json

import requests


def recorrer(data,listaValores,campo,bucle=True):
    for valor in data:
        if bucle:
            for valor2 in valor[campo]:
                if valor2 not in listaValores:
                    listaValores.append(valor2)
        else:
            listaValores.append(valor)
    return listaValores

def get_json(url):
     return json.loads(json.dumps((requests.get(url)).json()))

def get_apis_untilnext(json,list,text=None,recursive=False):
    while True:
        list=recorrer(json['results'],list,text,recursive)
        if(json['info']['next'] == None):
             break
        print(json["info"]["next"])
        json=get_json(json['info']['next'])
    return list

**Ejecución del buscador de páginas de location**

Aquí recogerá las páginas de la api de location hasta que no haya más

In [18]:
import threading


listaLocations=[]
listaResidentesTotal=[]
tEp=threading.Thread(target=get_apis_untilnext,args=[get_json("https://rickandmortyapi.com/api/location/"),listaLocations,])
tRe=threading.Thread(target=get_apis_untilnext,args=[get_json("https://rickandmortyapi.com/api/character/"),listaResidentesTotal,])
tEp.start()
tRe.start()
tEp.join()
tRe.join()

https://rickandmortyapi.com/api/location/?page=2
https://rickandmortyapi.com/api/character/?page=2
https://rickandmortyapi.com/api/character/?page=3
https://rickandmortyapi.com/api/location/?page=3
https://rickandmortyapi.com/api/character/?page=4
https://rickandmortyapi.com/api/location/?page=4
https://rickandmortyapi.com/api/character/?page=5
https://rickandmortyapi.com/api/location/?page=5
https://rickandmortyapi.com/api/character/?page=6
https://rickandmortyapi.com/api/location/?page=6
https://rickandmortyapi.com/api/character/?page=7
https://rickandmortyapi.com/api/location/?page=7
https://rickandmortyapi.com/api/character/?page=8
https://rickandmortyapi.com/api/character/?page=9
https://rickandmortyapi.com/api/character/?page=10
https://rickandmortyapi.com/api/character/?page=11
https://rickandmortyapi.com/api/character/?page=12
https://rickandmortyapi.com/api/character/?page=13
https://rickandmortyapi.com/api/character/?page=14
https://rickandmortyapi.com/api/character/?page=15


**Recogida de valores totales**

Aquí se recoge todos los valores de los residentes y teniendo en cuenta si no hay residente

Aclaración: Otro método que también se puede utilizar es guardando los personajes en una lista y comprobando el enlace y en vez de recorrer la api de cada personaje, recorrer la api de personajes para recoger cada uno de los personajes

In [19]:
import pandas as pd

def get_data(x,listaResidentesTotal):
    if x['residents'] is None:
        return {"id":x['id'],"name":x['name'],"type":x['type'],"dimension":x['dimension'],"resident1-name":"EMPTY NAME","resident1-species":"EMPTY SPECIES"}
    value=(x['residents'][0]) if len(x['residents'])>0 else None
    if value is None:
        return {"id":x['id'],"name":x['name'],"type":x['type'],"dimension":x['dimension'],"resident1-name":"EMPTY NAME","resident1-species":"EMPTY SPECIES"}
    value=list(filter(lambda x:int(x['id'])==int((value.split('/'))[-1]),listaResidentesTotal))
    return {"id":x['id'],"name":x['name'],"type":x['type'],"dimension":x['dimension'],"resident1-name":value[0]['name'],"resident1-species":value[0]['species']}


listaValores = list(map(lambda x:get_data(x,listaResidentesTotal),listaLocations))

**Transformación lista al DataFrame**

In [20]:
df=pd.DataFrame(listaValores)
df

,id,name,type,dimension,resident1-name,resident1-species
0,1,Earth (C-137),Planet,Dimension C-137,Beth Smith,Human
1,2,Abadango,Cluster,unknown,Abadango Cluster Princess,Alien
2,3,Citadel of Ricks,Space station,unknown,Adjudicator Rick,Human
3,4,Worldender's lair,Planet,unknown,Alan Rails,Human
4,5,Anatomy Park,Microverse,Dimension C-137,Alexander,Human
...,...,...,...,...,...,...
121,122,Avian Planet,Planet,Replacement Dimension,Alien Crow,Alien
122,123,Normal Size Bug Dimension,Dimension,,Palicki,Alien
123,124,Slartivart,Planet,Replacement Dimension,Slartivartian,Alien
124,125,Rick and Two Crows Planet,Planet,Replacement Dimension,Pussifer,Animal


# Ejercicio 3

Usaremos una ruta relativa a una carpeta de exportados ahí se guardarán los 3 archivos

In [21]:
import pathlib
from os import listdir
from os.path import isfile, join

path=str(pathlib.Path().resolve())+"/Exportados"
df.to_json(path+"/location.json")
df.to_csv(path+"/location.csv")
df.to_parquet(path+"/location.parquet")

**Recogida de uno de los ficheros**

En este caso utilizaremos el JSON para obtener del fichero guardado pero podríamos usar cualquiera adaptandolo

In [22]:
dfRead=pd.read_json(path+"/location.json")

# Ejercicio 4

## Operaciones con el DataFrame

Aquí recogeremos los planetas que sean de tipo **"Planet"** y cuantos hay

In [23]:
dfLocationsPlanet=dfRead[dfRead['type']=="Planet"]
print(f"TAMAÑO TOTAL: {dfLocationsPlanet.__len__()}")
dfLocationsPlanet

TAMAÑO TOTAL: 62


,id,name,type,dimension,resident1-name,resident1-species
0,1,Earth (C-137),Planet,Dimension C-137,Beth Smith,Human
3,4,Worldender's lair,Planet,unknown,Alan Rails,Human
7,8,Post-Apocalyptic Earth,Planet,Post-Apocalyptic Dimension,Armothy,unknown
8,9,Purge Planet,Planet,Replacement Dimension,Arthricia,Alien
9,10,Venzenulon 7,Planet,unknown,Beebo,Alien
...,...,...,...,...,...,...
111,112,Morglutz,Planet,Replacement Dimension,Morglutzian,Alien
112,113,Ferkus 9,Planet,Replacement Dimension,Daphne,Alien
121,122,Avian Planet,Planet,Replacement Dimension,Alien Crow,Alien
123,124,Slartivart,Planet,Replacement Dimension,Slartivartian,Alien


Aquí contaremos cuantas dimensiones diferentes hay

In [24]:
dfDimensiones = dfRead['dimension'].drop_duplicates()
print(f"HAY: {dfDimensiones.count()} dimensiones diferentes")

HAY: 34 dimensiones diferentes


En este tenemos que consultar las Dimensiones donde la localización tiene un id menor al de 100

In [25]:
dfDimensiones = dfRead[dfRead['id']<100]['dimension']
dfDimensiones

0           Dimension C-137
1                   unknown
2                   unknown
3                   unknown
4           Dimension C-137
              ...          
94    Replacement Dimension
95    Replacement Dimension
96    Replacement Dimension
97    Replacement Dimension
98    Replacement Dimension
Name: dimension, Length: 99, dtype: object

Y por último pide los planetas que empiecen por Earth

In [26]:
dfPlanetas2=dfRead['name']
dfPlanetas2=dfRead[dfRead['name'].str.contains('^(Earth)', regex=True)]['name']
dfPlanetas2

C:\Users\diego.guizanlopez\AppData\Local\Temp\ipykernel_9952\833260588.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dfPlanetas2=dfRead[dfRead['name'].str.contains('^(Earth)', regex=True)]['name']


0                                  Earth (C-137)
16                                 Earth (5-126)
19                 Earth (Replacement Dimension)
22                                Earth (C-500A)
25                                  Earth (K-83)
29                     Earth (Unknown dimension)
30                                 Earth (J19ζ7)
33          Earth (Evil Rick's Target Dimension)
41    Earth (Giant Telepathic Spiders Dimension)
51                                  Earth (K-22)
57                                  Earth (D-99)
58                                  Earth (D716)
59                                Earth (D716-B)
60                                Earth (D716-C)
61                                  Earth (J-22)
68                                  Earth (C-35)
70                       Earth (Pizza Dimension)
71                       Earth (Phone Dimension)
73                       Earth (Chair Dimension)
76                     Earth (Fascist Dimension)
79              Eart